# VICAV Bibliografie: Zotero Update

In [1]:
!ls

Untitled.ipynb
bibl_data.json
testing-zotero-down-up.ipynb
vicav-zotero-bib.ipynb
vicav_biblio_tei_zotero.xml
vicav_trash_20220523
vicav_update_zotero_library_Angelika.ipynb
vicav_update_zotero_library_Angelika_Evaluate_Library.ipynb
vicav_update_zotero_library_Angelika_Secondrun.ipynb


## Access Zotero-API

In [2]:
import requests
import json

In [3]:
api_token = "DQfd2PYuoDza5S0zMQdjzU6P"

In [4]:
#Group-id of Vicav library
group_id = "2165756"

In [5]:
#function to get the value of the "extra" field
def get_extra_field(zotero_id):
    request_url = "https://api.zotero.org/groups/" + group_id + "/items/" + zotero_id
    response = requests.get(request_url)
    if response.status_code == 200:
        parsed = json.loads(response.text)
        
        extra = parsed["data"]["extra"]
        
        if extra.startswith("(biblid:"):
            extra = extra.split("(biblid:")[1].split(")")[0]
    else: 
        extra = None        
    
    return extra

## Read all items in the library

In [17]:
def get_items(group_id,limit,start):
    """
    get zotero-items of a group, can download max.100 items in one run, need to set start and iterate to download whole library
    """
    request_url = "https://api.zotero.org/groups/" + group_id + "/items/" + "?limit=" + limit + "&start=" + start
    response = requests.get(request_url)
    if response.status_code == 200:
        parsed = json.loads(response.text)
        headers = response.headers
        
    return parsed,headers

## https://api.zotero.org/groups/2165756/items/

In [20]:
all_items=get_items("2165756","20","3")[0]

In [21]:
len(all_items)

20

In [12]:
def total_number_items(group_id):
    """
    get the number of total items of response header
    """
    request_url = "https://api.zotero.org/groups/" + group_id + "/items/"
    response = requests.get(request_url)
    
    return response.headers["Total-Results"]

In [47]:
total_number_items("2165756")

'6081'

In [14]:
def get_groupheaders(group_id):
    request_url = "https://api.zotero.org/groups/" + group_id + "/items/"
    response = requests.get(request_url)
    
    return response.headers
    

In [15]:
get_groupheaders("2165756")["Link"].split('; rel="next",')[0].replace("<","").replace(">","")

'https://api.zotero.org/groups/2165756/items/?start=25'

In [38]:
def get_links_from_headers(headers):
    link_list = headers["Link"].split(",")
    links = {}
    for link_item in link_list:
        #print(link_item)
        link_type = link_item.split('; rel="')[1].replace('"','').strip()
        link_value = link_item.split('; rel="')[0].replace("<","").replace(">","").strip()
        links[link_type] = link_value
    
    return links

        
        

In [40]:
get_links_from_headers(first_round[1])["next"]

'https://api.zotero.org/groups/2165756/items/?limit=100&start=101'

In [44]:
get_links_from_headers(first_round[1])

{'first': 'https://api.zotero.org/groups/2165756/items/?limit=50',
 'prev': 'https://api.zotero.org/groups/2165756/items/?limit=50',
 'next': 'https://api.zotero.org/groups/2165756/items/?limit=50&start=100',
 'last': 'https://api.zotero.org/groups/2165756/items/?limit=50&start=6050',
 'alternate': 'https://www.zotero.org/groups/2165756/items/'}

In [43]:
#test
allitems=[]
limit="50"
start="50"
#group_id
first_round=get_items(group_id,limit,start)
# print(first_round[0])
# print(first_round[1])
allitems=allitems+first_round[0]
next_url = get_links_from_headers(first_round[1])["next"]
print(next_url)


https://api.zotero.org/groups/2165756/items/?limit=50&start=100


In [51]:
allitems=[]
limit="100"
start="0"
first_round=get_items(group_id,limit,start)
allitems=allitems+first_round[0]
next_url = get_links_from_headers(first_round[1])["next"]
last_url = get_links_from_headers(first_round[1])["last"]
while next_url != last_url:
    print(next_url)
    response = requests.get(next_url)
    if response.status_code == 200:
        parsed = json.loads(response.text)
        headers = response.headers
        
        allitems=allitems + parsed
        urls = get_links_from_headers(headers)
        if "next" in urls:
            next_url = urls["next"]
        else:
            break
    else:
        break
print(last_url)
response = requests.get(last_url)
if response.status_code == 200:
    parsed = json.loads(response.text)
    allitems=allitems + parsed
print("Done!")

https://api.zotero.org/groups/2165756/items/?limit=100&start=100
https://api.zotero.org/groups/2165756/items/?limit=100&start=200
https://api.zotero.org/groups/2165756/items/?limit=100&start=300
https://api.zotero.org/groups/2165756/items/?limit=100&start=400
https://api.zotero.org/groups/2165756/items/?limit=100&start=500
https://api.zotero.org/groups/2165756/items/?limit=100&start=600
https://api.zotero.org/groups/2165756/items/?limit=100&start=700
https://api.zotero.org/groups/2165756/items/?limit=100&start=800
https://api.zotero.org/groups/2165756/items/?limit=100&start=900
https://api.zotero.org/groups/2165756/items/?limit=100&start=1000
https://api.zotero.org/groups/2165756/items/?limit=100&start=1100
https://api.zotero.org/groups/2165756/items/?limit=100&start=1200
https://api.zotero.org/groups/2165756/items/?limit=100&start=1300
https://api.zotero.org/groups/2165756/items/?limit=100&start=1400
https://api.zotero.org/groups/2165756/items/?limit=100&start=1500
https://api.zotero.

In [54]:
len(allitems)

6081

In [53]:
allitems[0]

{'key': 'JAAFVQTT',
 'version': 17033,
 'library': {'type': 'group',
  'id': 2165756,
  'name': 'VICAV',
  'links': {'alternate': {'href': 'https://www.zotero.org/groups/vicav',
    'type': 'text/html'}}},
 'links': {'self': {'href': 'https://api.zotero.org/groups/2165756/items/JAAFVQTT',
   'type': 'application/json'},
  'alternate': {'href': 'https://www.zotero.org/groups/vicav/items/JAAFVQTT',
   'type': 'text/html'}},
 'meta': {'createdByUser': {'id': 4438625,
   'username': 'Terlan',
   'name': '',
   'links': {'alternate': {'href': 'https://www.zotero.org/terlan',
     'type': 'text/html'}}},
  'lastModifiedByUser': {'id': 6961982,
   'username': 'anairiarted',
   'name': '',
   'links': {'alternate': {'href': 'https://www.zotero.org/anairiarted',
     'type': 'text/html'}}},
  'creatorSummary': 'Eisele',
  'parsedDate': '1997',
  'numChildren': 0},
 'data': {'key': 'JAAFVQTT',
  'version': 17033,
  'itemType': 'journalArticle',
  'title': 'On the Description and Representation o

## Check bibl-id

In [101]:
bibl_data = []
missing = []
attachments = []
notes = []
for bibl in allitems:
    item = {}
    item["zotero_url"] = bibl["links"]["self"]["href"]
    item["xml_id"] = None
    item["zotero_id"] = bibl["key"]
    #check extra
    if "extra" in bibl["data"]:
        item["extra"] = bibl["data"]["extra"]
    
    if bibl["data"]["itemType"] == "attachment":
        attachments.append(item)
    elif bibl["data"]["itemType"] == "note":
        notes.append(item)
    else:
        if item["extra"] == "":
            missing.append(item)
        
        bibl_data.append(item)
    

In [102]:
len(bibl_data)

4990

In [103]:
len(missing)

268

In [88]:
bibl_data[0]

{'zotero_url': 'https://api.zotero.org/groups/2165756/items/JAAFVQTT',
 'xml_id': None,
 'zotero_id': 'JAAFVQTT',
 'extra': '(biblid:eisele_1997_3476)'}

In [89]:
bibl_data[333]

{'zotero_url': 'https://api.zotero.org/groups/2165756/items/RSVR9HM9',
 'xml_id': None,
 'zotero_id': 'RSVR9HM9',
 'extra': '(biblid:miller_2009_6039)'}

In [104]:
len(missing)

268

In [105]:
missing[0]

{'zotero_url': 'https://api.zotero.org/groups/2165756/items/H5Z6QYGF',
 'xml_id': None,
 'zotero_id': 'H5Z6QYGF',
 'extra': ''}

In [106]:
len(notes)

1045

In [107]:
len(attachments)

46

In [112]:
len(notes) + len(attachments) + len(bibl_data) 

6081

In [75]:
attachments[0]

{'zotero_url': 'https://api.zotero.org/groups/2165756/items/3JX68A6E',
 'xml_id': None,
 'zotero_id': '3JX68A6E'}

In [76]:
attachments[1]

{'zotero_url': 'https://api.zotero.org/groups/2165756/items/FJD2FHSD',
 'xml_id': None,
 'zotero_id': 'FJD2FHSD'}

## Export list with all bibl-data extra

In [78]:
import csv

f = open('bibldata_vicav.csv', 'w')
writer = csv.writer(f)

for item in bibl_data:
    row = []
    row.append(item["zotero_url"])
    row.append(item["zotero_id"])
    if "extra" in item:
        row.append(item["extra"])
    else:
        row.append("")
    
    writer.writerow(row)
f.close()

## Update missing extra-fields

In [80]:
api_token = "DQfd2PYuoDza5S0zMQdjzU6P"
headers = {'Authorization': 'Bearer ' + api_token}

In [81]:
# Function to update zotero-item

def update_zotero_extra_in_item(url:str, extra:str):
    #test_url = "https://api.zotero.org/groups/4669401/items/P4TV9RFP"
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        parsed = json.loads(response.text)
        
        if "extra" in parsed["data"]:
            old_extra_val = parsed["data"]["extra"]
            #print("Existing 'extra' field value was: " + old_extra_val)
            
            #updating
            parsed["data"]["extra"] = extra
            
            update = requests.put(url, headers=headers, json=parsed)
            if update.status_code == 204:
                #everything worked
                #return True as idicator, that everything worked as expected, and the old "extra" value (could be stored somewhere for safe-keeping)
                return [True,old_extra_val]
            else:
                print("Error when updating item. Status code:" + str(update.status_code))
        
    else:
        print("Error when getting item. Status code: " + str(response.status_code))    
    

In [127]:
import re
import random

In [130]:
## create extra value
def create_extra_val(url:str):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        parsed = json.loads(response.text)
        
        #last name first creator
        lastname = parsed["data"]["creators"][0]["lastName"]
        pubyear = parsed["data"]["date"]
        #print("Name: " + name + " Year: " + year + " Suffix: " + suffix )
        r = random.randint(1,9999)
        randnr = "%04d" % r
        new_id = lastname.lower() + "_" + pubyear + "_" + randnr
        
        
        return new_id

In [131]:
create_extra_val(missing[0]["zotero_url"])

'al-jallad_2021_5878'

In [134]:
still_errors=[]
counter=1
for item in missing:
    print(str(counter) + " von " + str(len(missing)))
    print("Updating " + item["zotero_id"])
    try:
        new_id = create_extra_val(item["zotero_url"])
    except:
        new_id = None
        print("error to generate id: " + item["zotero_id"])
        still_errors.append(item)
    
    if new_id != None:
        new_extra = "(" + "biblid:" + new_id + ")"
        item["biblid"] = new_extra
        updated = update_zotero_extra_in_item(item["zotero_url"], new_extra)
        item["updated"] = updated[0]
        item["old_extra_val"] = updated[1]
    counter = counter + 1

1 von 268
Updating H5Z6QYGF
2 von 268
Updating RD9G436Q
3 von 268
Updating NGK4FLL9
4 von 268
Updating DG6ITECY
5 von 268
Updating 2JQQSW26
6 von 268
Updating HX8QK5DH
7 von 268
Updating 8TFNC9N4
8 von 268
Updating X9BYVAM5
9 von 268
Updating ENYPZHMC
10 von 268
Updating UPV33IP6
11 von 268
Updating BBDE2SVD
12 von 268
Updating UBUNBJZH
13 von 268
Updating 8FUJQ2QM
14 von 268
Updating IIS2SV4G
15 von 268
Updating 5BIJV6W5
16 von 268
Updating PBL27MRR
17 von 268
Updating UZARQZGV
18 von 268
Updating ZKXZETIR
19 von 268
Updating 5XMCGUMW
20 von 268
Updating GV4MQQJW
21 von 268
Updating 5VR4DBB7
22 von 268
Updating R8F3CRX6
23 von 268
Updating 48XB7LED
24 von 268
Updating H49IUL33
25 von 268
Updating 73V9CH52
26 von 268
Updating 2S4VPJZX
27 von 268
Updating UKTLGGAF
28 von 268
Updating 5LYW9HD4
29 von 268
Updating 3MB7YTPC
30 von 268
Updating ABSSGZSI
31 von 268
Updating T5CDYBDV
32 von 268
Updating KBTFRP8Z
33 von 268
Updating I23G826D
34 von 268
Updating 6PVPCANE
35 von 268
Updating IL8

In [135]:
still_errors

[{'zotero_url': 'https://api.zotero.org/groups/2165756/items/PCSVHHES',
  'xml_id': None,
  'zotero_id': 'PCSVHHES',
  'extra': ''},
 {'zotero_url': 'https://api.zotero.org/groups/2165756/items/5VSJEDQ8',
  'xml_id': None,
  'zotero_id': '5VSJEDQ8',
  'extra': ''},
 {'zotero_url': 'https://api.zotero.org/groups/2165756/items/ZJNYEVY9',
  'xml_id': None,
  'zotero_id': 'ZJNYEVY9',
  'extra': ''},
 {'zotero_url': 'https://api.zotero.org/groups/2165756/items/QUM4S3AG',
  'xml_id': None,
  'zotero_id': 'QUM4S3AG',
  'extra': ''},
 {'zotero_url': 'https://api.zotero.org/groups/2165756/items/QXMC4VJF',
  'xml_id': None,
  'zotero_id': 'QXMC4VJF',
  'extra': ''},
 {'zotero_url': 'https://api.zotero.org/groups/2165756/items/U3GMANW4',
  'xml_id': None,
  'zotero_id': 'U3GMANW4',
  'extra': ''}]